In [1]:
import os
import numpy as np
import tensorflow as tf

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
matplotlib.rc('font', family='AppleGothic')  # MacOS
# matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False

C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (

In [2]:
# 9.1 설치
import tensorflow as tf

tf.__version__

'1.13.1'

In [3]:
# 9.2 첫 번째 계산 그래프를 만들어 세션에서 실행하기
import tensorflow as tf

reset_graph()

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2
f

Instructions for updating:
Colocations handled automatically by placer.


<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [5]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

# Session 닫기
sess.close()

42


In [6]:
# with 스코프를 이용해 간단하게 작성
# with 스코프가 끝나면 tf.Session은 자동 종료
with tf.Session() as sess:
    x.initializer.run()  # tf.get_default_session().run(x.initializer)
    y.initializer.run()
    result = f.eval()  # tf.get_default_session().run(f)
    
print(result)

42


In [7]:
# 9.3 계산 그래프 관리
# 노드를 만들면 자동으로 기본 계산 그래프에 추가된다.
reset_graph()

x1 = tf.constant(1)
x1.graph is tf.get_default_graph()

True

In [8]:
# tf.Graph()를 이용해 새로운 독립적인 계산 그래프를 만들 수 있다.
# as_default()를 통해 기본 계산 그래프로 사용할 수 있다.
graph = tf.Graph()
with graph.as_default():
    x2 = tf.constant(2)
    print(x2.graph is tf.get_default_graph())  # True
    
print(x2.graph is tf.get_default_graph())  # False
print(x2.graph is graph)  # True

True
False
True


In [9]:
# 9.4 노드 값의 생애주기
# 각 그래프에는 노드 간에 의존관계(dependency)가 존재한다
# 직접의존 간접의존 구분

w = tf.constant(3)
x = w + 2
y = tf.add(x, 5)
z = x * 3

with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)  # 10
    print(z_val)  # 15

10
15


In [10]:
# 9.5 텐서플로를 이용한 선형 회귀
# 정규식을 이용한 선형 회귀 풀이
import numpy as np
from sklearn.datasets import fetch_california_housing

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
# bias 추가
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

print('data shape :', housing.data.shape)
print('data_plus_bias shape :', housing_data_plus_bias.shape)

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')  # (20640, 1)
X_t = tf.transpose(X)
weight = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(X_t, X)), X_t), y)

with tf.Session() as sess:
    weight_val = weight.eval()
    
print(weight_val)

data shape : (20640, 8)
data_plus_bias shape : (20640, 9)
[[-3.6289421e+01]
 [ 4.3852323e-01]
 [ 9.5494352e-03]
 [-1.0940705e-01]
 [ 6.5344292e-01]
 [-3.6061101e-06]
 [-3.8037887e-03]
 [-4.1441277e-01]
 [-4.2695773e-01]]


In [11]:
# Scikit-Learn과 비교
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [18]:
# 9.6 경사 하강법 구현
# 경사 하강법을 사용할 때는 입력 특성 벡터를 정규화(normalization)해주는
# 것이 중요하다. 그렇지 않으면 학습속도가 매우 느려진다.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [ ]:
# 9.6.1 직접 그래디언트 계산

In [ ]:
# 9.6.2 자동 미분 사용

In [ ]:
# 9.6.3 Optimizer 사용하기

### 9.7 훈련 알고리즘에 데이터 주입
Placeholder
딥러닝에서 데이터에 대한 학습이 이루어질 때 학습할 데이터들을 입력해줘야 한다. 텐서플로에서는 입력값을 넣어주기 위해 플레이스홀더(placeholder)라는 것이 있다. 플레이스홀더는 데이터를 입력받는 비어있는 변수라고 생각할 수 있다. 먼저 그래프를 구성하고, 그 그래프가 실행되는 시점에 입력 데이터를 넣어주는 데 사용한다.

플레이스홀더는 shape 인수를 유동적으로 지정할 수 있다. 예를 들어, None으로 지정되면 이 플레이스홀더는 모든 크기의 데이터를 받을 수 있다. 주로 배치단위(batch size)의 샘플 데이터 개수에 해당 되는 부분(데이터의 행)은 None을 사용하고, 데이터 Feature의 길이(데이터의 열)는 고정된 값을 사용한다.

플레이스홀더를 정의하면 반드시 그래프 실행 단계에서 입력값을 넣어줘야 하며, 그렇지 않을 경우 에러가 나타난다. 입력 데이터는 딕셔너리(dictionary)형태로 session.run()메소드를 통해 전달된다. 딕셔너리의 키(key)는 플레이스홀더 변수 이름에 해당하며 값(value)은 list 또는 NumPy 배열이다.

In [23]:
reset_graph()

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: np.array([[4, 5, 6],
                                             [7, 8, 9]])})
    
print('B_val_1:\n{}'.format(B_val_1))
print('B_val_2:\n{}'.format(B_val_2))

B_val_1:
[[6. 7. 8.]]
B_val_2:
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [24]:
# Mini-batch Gradient Descent
reset_graph()

# placeholder setting
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")  # (None, 9)
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

W = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='W')

y_pred = tf.matmul(X, W, name='predictions')  # y = wx + b
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

################
# hyper-params #
################
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))  # (20640/100) 올림

# mini-batch random sampling
def fetch_batch(epoch, batch_index, batch_size):
    """ epoch당 batch_size만큼 Random sampling 하는 메소드
    
    Args:
        - epoch: training epoch
        - batch_index: batch steps
        - batch_size: batch size
    
    Returns:
        - X_batch, y_batch: batch size만큼의 X, y 데이터셋
    """    
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch


# Training
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    best_W = W.eval()
    
print(best_W)

NameError: name 'learning_rate' is not defined

In [ ]:
# 9.8 모델 저장과 복원

### 9.9 TensorBoard로 그래프와 학습 곡선 시각화하기
tfgraphviz 모듈을 이용한 시각화
$ pip install graphviz # or conda install graphviz
$ pip install tfgraphviz

In [27]:
import tfgraphviz as tfg

tf_graph = tfg.board(tf.get_default_graph())
tf_graph

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH

### 9.10 Name Scope
복잡한 그래프를 처리해야 하는 경우 이를 쉽게 추적하고 관리하기 위해서 노드를 이름별로 그룹화하여 묶는 것이 편리하다. with구문과 tf.name_scope('prefix')를 사용하면 이름 스코프를 사용할 수 있다.

아래의 예제는 c2, c3를 prefix_name이라는 스코프로 그룹화한 것이다. 출력결과를 보면 텐서 객체의 이름 앞에 접두사 형태로 prefix_name이 붙은 것을 알 수 있다.

접두사는 그래프를 의미에 따라 서브그래프로 나누고자 할 때 유용하며, 그래프의 구조를 시각화할 때 사용할 수 있다.

In [19]:
with tf.Graph().as_default():
    c1 = tf.constant(4, dtype=tf.float64, name='c')
    with tf.name_scope("prefix_name"):
        c2 = tf.constant(4, dtype=tf.int32, name='c')
        c3 = tf.constant(4, dtype=tf.float64, name='c')

print('c1.name >>', c1.name)
print('c2.name >>', c2.name)
print('c3.name >>', c3.name)

c1.name >> c:0
c2.name >> prefix_name/c:0
c3.name >> prefix_name/c_1:0


### 9.11 모듈화
텐서플로를 이용해 딥러닝 모델링을 하다보면 같은 작업을 수행하는 부분이 많다는 것을 알 수 있다. 아래의 예제를 보자

In [20]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

In [21]:
# 위의 중복이 발생하는 부분을 relu() 함수를 이용해 간단하게 표현할 수 있다.
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                          
        w = tf.Variable(tf.random_normal(w_shape), name="weights")    
        b = tf.Variable(0.0, name="bias")                             
        z = tf.add(tf.matmul(X, w), b, name="z")                      
        return tf.maximum(z, 0., name="max")
    
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [22]:
tf_graph = tfg.board(tf.get_default_graph())
tf_graph

NameError: name 'tfg' is not defined